<h2>개인 구글 드라이브와 colab 연동 </h2>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


<h2>라이브러리 설치 </h2>

In [3]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 747kB 31.3MB/s 


<pre>
<h2> 1. "spacing_data.txt"로 부터 데이터를 읽고 datas에 저장 </h2>
  1.1 입력 데이터 형태
    예시) 나 는 사 과 가 좋 아 \t B I B I I B I
    
    "나는 사과가 좋아" 라는 문장이 있을 때, 공백을 기준으로 나눈 "나는", "사과가", "좋아"를 각각 어절이라고 함
    각 어절의 시작음절을 "B"로 나머지를 "I"로 태깅하였고 문장과 라벨을 \t으로 구분해놓은 상태

  1.2 데이터 변환

    입력 데이터를 읽고 탭을 기준으로 문장과 라벨을 분리
    문장을 공백을 기준으로 나눈다
      예시)"나 는 사 과 가 좋 아" -> ["나", "는", "사", "과", "가", "좋", "아"]
    문장과 라벨을 튜플형태로 datas 리스트에 넣는다
    datas = [ ( ["나", "는", "사", "과", "가", "좋", "아"], ["B", "I", "B", "I", "I", "B", "I"] ), ( ... ), ... ]
    
<h2> 2. 전체 데이터를 9:1 비율에 맞추어 학습, 평가 데이터로 나누기 </h2>
  2.1 train_datas 리스트와 test_datas 리스트에 나누어 저장
</pre>

In [4]:
import os
import sklearn_crfsuite
from sklearn_crfsuite import metrics


# 파일 경로
file_path = "/gdrive/My Drive/colab/crf/spacing_data.txt"

# "spacing_data.txt" 파일을 읽고 lines에 읽은 데이터를 저장
with open(file_path, "r", encoding="utf8") as inFile:
    lines = inFile.readlines()

# 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
datas = []
for line in lines:
    pieces = line.strip().split("\t")
    eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
    datas.append((eumjeol_sequence, label))
    
number_of_train_datas = int(len(datas)*0.9)

train_datas = datas[:number_of_train_datas]
test_datas = datas[number_of_train_datas:]
          
print("train_datas 개수 : " + str(len(train_datas)))          
print("test_datas 개수 : " + str(len(test_datas)))

train_datas 개수 : 900
test_datas 개수 : 100


<pre>
<h2> 1. 문장의 각 음절을 crf 모델의 입력으로 사용 할 수 있도록 자질화 </h2>
  "BOS" : 시작 음절인지 여부
  "EOS" : 마지막 음절인지 여부
  "WORD" : 기준 음절
  "IS_DIGIT" : 기준 음절이 숫자인지 여부
  "-1_WORD" : 기준 음절의 왼쪽 첫번째 음절
  "-2_WORD" : 기준 음절의 왼쪽 두번째 음절
  "+1_WORD" : 기준 음절의 오른쪽 첫번째 음절
  "+2_WORD" : 기준 음절의 오른쪽 두번째 음절

  1.1 예시) ["나", "는", "사", "과", "가", "좋", "아"]
            ->
            [ { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" },
    { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" },
    { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }, ... ]
    
    나 -> { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" }
    는 -> { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" }
    사 -> { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }
    과 -> { "BOS":False, "EOS":False, "WORD":"과", "IS_DIGIT":False, "-2_WORD":"는", "-1_WORD":"사", "+1_WORD":"가", "+2_WORD":"좋" }
    가 -> { "BOS":False, "EOS":False, "WORD":"가", "IS_DIGIT":False, "-2_WORD":"사", "-1_WORD":"과", "+1_WORD":"좋", "+2_WORD":"아" }
    좋 -> { "BOS":False, "EOS":False, "WORD":"좋", "IS_DIGIT":False, "-2_WORD":"과", "-1_WORD":"가", "+1_WORD":"아" }
    아 -> { "BOS":False, "EOS":True, "WORD":"아", "IS_DIGIT":False, "-2_WORD":"가", "-1_WORD":"좋" }
<h2> 2. 자질화한 데이터와 해당 데이터의 라벨을 분리하여 각 리스트에 저장 </h2>
  학습 데이터 -> train_x(자질화한 데이터), train_y(각 데이터의 정답 라벨)에 저장
  평가 데이터 -> test_x(자질화한 데이터), test_y(각 데이터의 정답 라벨)에 저장
  
  2.1 예시)
    train_x -> [
    
    [ { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" },
    { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" },
    { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }, ... ],
    
    [ ... ],
    
    ...
    
    ]
    
    train_y -> [
    
    [ "B", "I", "B", "I", "I", "B", "I" ],
    
    [ ... ],
    
    ...
    
    
    ]

</pre>

In [6]:
def sent2feature(eumjeol_sequence):
  features = []
  sequence_length = len(eumjeol_sequence)
  for index, eumjeol in enumerate(eumjeol_sequence):
      feature = { "BOS":False, "EOS":False, "WORD":eumjeol, "IS_DIGIT":eumjeol.isdigit() }

      if(index == 0):
          feature["BOS"] = True
      elif(index == sequence_length-1):
          feature["EOS"] = True

      if(index-1 >= 0):
          feature["-1_WORD"] = eumjeol_sequence[index-1]
      if(index-2 >= 0):
          feature["-2_WORD"] = eumjeol_sequence[index-2]

      if(index+1 <= sequence_length-1):
          feature["+1_WORD"] = eumjeol_sequence[index+1]
      if(index+2 <= sequence_length-1):
          feature["+2_WORD"] = eumjeol_sequence[index+2]

      features.append(feature)

  return features
  
  
train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

<h2> 1. train_x, train_y를 이용하여 crf 모델 학습 </h2>

In [16]:
#crf = sklearn_crfsuite.CRF(
#        algorithm='lbfgs',
#        c1=0.1,
#        c2=0.1,
#        max_iterations=100,
#        all_possible_transitions=True
#    )
crf = sklearn_crfsuite.CRF()
crf.fit(train_x, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm=None, all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

<pre>
<h2> 1. 학습한 모델을 test_x 데이터를 사용하여 평가 </h2>
<h2> 2. 성능 측정 </h2>
  2.1 metrics.flat_accuracy_score(x, y) 함수를 이용하여 성능 측정
    metrics.flat_accuracy_score(x, y)
  args
    x : 실제 정답 라벨이 있는 리스트
    y : 모델의 출력 라벨이 있는 리스트
  return : 
    accuract 성능
  
<h2> 3. 모델의 출력 값과 정답 값을 이용하여 음절만으로 구성된 완전한 문장으로 변형 </h2>
  3.1 test_datas, pred_y 예시
    test_datas = [
    ( ["나", "는", "사", "과", "가", "좋", "아"], ["B", "I", "B", "I", "I", "B", "I"] ),
    
    ( ... ),
    
    ...
    
    ]
    
    pred_y = [
    
    ["B", "B", "B", "I", "I", "I", "I"],
    
    [ ... ],
    
    ...
    
    ]
    
    위의 문장을 기준으로한 변형 예시
    
    "나", "는", "사", "과", "가", "좋", "아" -> 나는 사과가 좋아 (정답 기준으로 변형)
    "나", "는", "사", "과", "가", "좋", "아" -> 나 는 사과가좋아 (모델 출력 기준으로 변형)
  
  3.2 최종 출력 예시
  
    정답 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
    출력 문장 : 19 14- 18년의 전쟁은 인류를 통합시킨 최초의 공통 분모였다.

    정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
    출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

    정답 문장 : 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
    출력 문장 : 사라 예보에서 한세르비아인이 쏜총한 발이 합스부르크가의 계승자를 죽였다.
    
    ...
  
  
</pre>


In [15]:
def show_predict_result(test_datas, predict):
  for index_1 in range(len(test_datas)):
      eumjeol_sequence, correct_labels = test_datas[index_1]
      predict_labels = predict[index_1]
     
      correct_sentence, predict_sentence = "", ""
      for index_2 in range(len(eumjeol_sequence)):
          if(index_2 == 0):
              correct_sentence += eumjeol_sequence[index_2]
              predict_sentence += eumjeol_sequence[index_2]
              continue

          if(correct_labels[index_2] == "B"):
              correct_sentence += " "
          correct_sentence += eumjeol_sequence[index_2]

          if (predict_labels[index_2] == "B"):
              predict_sentence += " "
          predict_sentence += eumjeol_sequence[index_2]

      print("정답 문장 : " + correct_sentence)
      print("출력 문장 : " + predict_sentence)
      print()

predict = crf.predict(test_x)

print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, predict)))
print()

print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:10], predict[:10])

Accuracy score : 0.8964135826020603

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
출력 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분 모였다.

정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

정답 문장 : 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
출력 문장 : 사라 예보에서 한 세르 비아인이 쏜총한 발이 합스부르크가의 계승 자를 죽였다.

정답 문장 : 이 암살행위는 국지적인 민족주의들과 세계적인 제국주의들이 충돌하는 분쟁지역에서 저질러졌다.
출력 문장 : 이암 살행 위는 국지적인 민족주의 들과 세계적인 제국주의 들이 충돌하는 분쟁 지역에서 저질러졌다.

정답 문장 : 오토만제국의 점진적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.
출력 문장 : 오토만 제국의 점진 적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.

정답 문장 : 이렇게 해서 발칸 반도의 한 외진 장소에서 벌어진 국지적인 테러 행위는 일련의 긴박한 반응을 불러 일으키면서 전 유럽에 영향을 미쳤을 뿐만 아니라 이번에는 아시아와 아프리카 식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.
출력 문장 : 이렇게 해서 발칸 반도의 한외진 장소에서 벌어 진국지적인 테러행 위는 일련의 긴박한 반응을 불러일으키면서 전유럽에 영향을 미쳤을 뿐만 아니라이 번에는 아시아와 아프리카 식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.

정답 문장 : 전쟁의 물결이 지구상의 모든 대양으로 밀려드는 동안 캐나다인들과 미국인들, 오스트레일리아인들, 세네갈인들, 알제리인들, 모로코인들, 안남(安南)인들은 연합군 깃발을 휘날리며 유럽전